In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lab-7-part2/name_variations.csv
/kaggle/input/lab-7-part2/base_names.csv


In [2]:
import pandas as pd
from fuzzywuzzy import fuzz

/opt/conda/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
name_variations = pd.read_csv('/kaggle/input/lab-7-part2/name_variations.csv', header=None, names=['Name_Variation'])
base_names = pd.read_csv('/kaggle/input/lab-7-part2/base_names.csv', header=None, names=['Base_Name'])

In [4]:
name_variations

,Name_Variation
Variation,Matches_With_Base_Name
Thomas King,Thomas King
ThomasKing,Thomas King
Maria Garcia,Maria Garcia
MaryLewis,Mary Lewis
...,...
Jennifer- Brown,Jennifer Brown
Daniel- Scott,Daniel Scott
David M.,David Martinez
Paul Allen.,Paul Allen


In [5]:
base_names

,Base_Name
Base_Name_ID,Base_Name
1,John Smith
2,Jennifer Brown
3,Michael O'Connor
4,Maria Garcia
5,Robert Lee
6,Linda Johnson
7,William Davis
8,Elizabeth Wilson
9,David Martinez


In [6]:
# Preprocess the names: lowercasing and stripping extra spaces
name_variations['Processed_Name_Variation'] = name_variations['Name_Variation'].str.lower().str.strip()
base_names['Processed_Base_Name'] = base_names['Base_Name'].str.lower().str.strip()

In [7]:
# Function to match name variations with base names using fuzzy matching
def match_names(variation_name, base_names):
    # Apply token set ratio fuzzy matching
    match_score = base_names['Processed_Base_Name'].apply(lambda x: fuzz.token_set_ratio(variation_name, x))
    
    # Find the index of the best match
    best_match_idx = match_score.idxmax()
    
    # Get the highest score and corresponding base name
    best_match_score = match_score.max()
    best_match_name = base_names.iloc[best_match_idx]['Base_Name']
    
    return best_match_name, best_match_score

In [8]:
# Ensure the indices are reset
base_names = base_names.reset_index(drop=True)
name_variations = name_variations.reset_index(drop=True)

# Apply the matching function to the name variations
name_variations['Matched_Base_Name'], name_variations['Match_Score'] = zip(
    *name_variations['Processed_Name_Variation'].apply(lambda x: match_names(x, base_names))
)

# Display the results
print("\nMatching Results:")
print(name_variations[['Name_Variation', 'Matched_Base_Name', 'Match_Score']].head())
# Ensure the indices are reset
base_names = base_names.reset_index(drop=True)
name_variations = name_variations.reset_index(drop=True)

# Apply the matching function to the name variations
name_variations['Matched_Base_Name'], name_variations['Match_Score'] = zip(
    *name_variations['Processed_Name_Variation'].apply(lambda x: match_names(x, base_names))
)

# Display the results
print("\nMatching Results:")
print(name_variations[['Name_Variation', 'Matched_Base_Name', 'Match_Score']])


Matching Results:
           Name_Variation Matched_Base_Name  Match_Score
0  Matches_With_Base_Name         Base_Name           58
1             Thomas King       Thomas King          100
2             Thomas King       Thomas King          100
3            Maria Garcia      Maria Garcia          100
4              Mary Lewis        Mary Lewis          100

Matching Results:
             Name_Variation Matched_Base_Name  Match_Score
0    Matches_With_Base_Name         Base_Name           58
1               Thomas King       Thomas King          100
2               Thomas King       Thomas King          100
3              Maria Garcia      Maria Garcia          100
4                Mary Lewis        Mary Lewis          100
..                      ...               ...          ...
96           Jennifer Brown    Jennifer Brown          100
97             Daniel Scott      Daniel Scott          100
98           David Martinez    David Martinez          100
99               Paul Allen   